In [1]:
# 라이브러리 임포트
import os
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import numpy as np
from collections import Counter
from sklearn.metrics import accuracy_score, f1_score
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import joblib
from Model import Transformer, PositionalEncoding
from TrainDataset import prepare_classification_dataset, tensor_dataset

In [2]:
df = pd.read_csv(f'datasets/sentiment_train.csv', index_col=0)

# '감정' 레이블을 숫자로 매핑
df.loc[(df['감정'] == '불안'), '감정'] = 0
df.loc[(df['감정'] == '당황'), '감정'] = 1
df.loc[(df['감정'] == '분노'), '감정'] = 2
df.loc[(df['감정'] == '슬픔'), '감정'] = 3
df.loc[(df['감정'] == '중립'), '감정'] = 4
df.loc[(df['감정'] == '행복'), '감정'] = 5
df.loc[(df['감정'] == '혐오'), '감정'] = 6

print(f"원본 df 크기: {len(df)}")
print(f"원본 df 감정 분포 (매핑 후): {Counter(df['감정'])}")

train_df, val_df = train_test_split(df, train_size=0.8, test_size=0.2, stratify=df['감정'], random_state=42) # 재현성을 위해 random_state 추가

print(f"학습 데이터프레임 크기: {len(train_df)}")
print(f"검증 데이터프레임 크기: {len(val_df)}")
print(f"학습 데이터프레임 감정 분포: {Counter(train_df['감정'])}")
print(f"검증 데이터프레임 감정 분포: {Counter(val_df['감정'])}")


print("학습 데이터 파싱 중...")
train_datasets_dict = prepare_classification_dataset(train_df)
print("검증 데이터 파싱 중...")
val_datasets_dict = prepare_classification_dataset(val_df)

train_datasets = tensor_dataset(train_datasets_dict)
val_datasets = tensor_dataset(val_datasets_dict)

print(f"학습 데이터셋 크기: {len(train_datasets)}")
print(f"검증 데이터셋 크기: {len(val_datasets)}")

BATCH_SIZE = 32
train_loader = DataLoader(
    train_datasets,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=0
)
print(f"학습 DataLoader 배치 수: {len(train_loader)}")

val_loader = DataLoader(
    val_datasets,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0
)
print(f"검증 DataLoader 배치 수: {len(val_loader)}")

VOCAB_SIZE = 32000
MAX_LEN =128
OUTPUT_DIM = int(df['감정'].nunique())
PAD_TOKEN_ID = 0

print(f"\n설정된 전역 변수:")
print(f"  VOCAB_SIZE: {VOCAB_SIZE}")
print(f"  MAX_LEN: {MAX_LEN}")
print(f"  OUTPUT_DIM: {OUTPUT_DIM}")
print(f"  PAD_TOKEN_ID: {PAD_TOKEN_ID}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\n사용 가능한 장치: {device}")

VERSION_FILE_PATH = os.path.join('saves', 'optuna_version.txt')

OPTUNA_LOG_MODEL_ROOT_DIR = 'optuna_runs'

def get_next_version():
    os.makedirs('saves', exist_ok=True) 
    current_version = 0
    if os.path.exists(VERSION_FILE_PATH):
        with open(VERSION_FILE_PATH, 'r') as f:
            try:
                current_version = int(f.read().strip())
            except ValueError:
                current_version = 0
    next_version = current_version + 1
    with open(VERSION_FILE_PATH, 'w') as f:
        f.write(str(next_version))
    return next_version

CURRENT_OPTUNA_VERSION = get_next_version()
print(f"\nOptuna 최적화 버전: v{CURRENT_OPTUNA_VERSION} (버전 파일: {VERSION_FILE_PATH})")

OPTUNA_VERSION_DIR = os.path.join(OPTUNA_LOG_MODEL_ROOT_DIR, f'v{CURRENT_OPTUNA_VERSION}')
os.makedirs(OPTUNA_VERSION_DIR, exist_ok=True)
print(f"모든 Optuna Trial 로그 및 모델은 '{OPTUNA_VERSION_DIR}' 아래에 저장됩니다.")

원본 df 크기: 88110
원본 df 감정 분포 (매핑 후): Counter({4: 48501, 2: 9265, 3: 7221, 5: 7054, 6: 5642, 0: 5565, 1: 4862})
학습 데이터프레임 크기: 70488
검증 데이터프레임 크기: 17622
학습 데이터프레임 감정 분포: Counter({4: 38801, 2: 7412, 3: 5777, 5: 5643, 6: 4513, 0: 4452, 1: 3890})
검증 데이터프레임 감정 분포: Counter({4: 9700, 2: 1853, 3: 1444, 5: 1411, 6: 1129, 0: 1113, 1: 972})
학습 데이터 파싱 중...


데이터 파싱 중:   0%|          | 0/70488 [00:00<?, ?it/s]

검증 데이터 파싱 중...


데이터 파싱 중:   0%|          | 0/17622 [00:00<?, ?it/s]

학습 데이터셋 크기: 70488
검증 데이터셋 크기: 17622
학습 DataLoader 배치 수: 2203
검증 DataLoader 배치 수: 551

설정된 전역 변수:
  VOCAB_SIZE: 32000
  MAX_LEN: 128
  OUTPUT_DIM: 7
  PAD_TOKEN_ID: 0

사용 가능한 장치: cuda

Optuna 최적화 버전: v7 (버전 파일: saves\optuna_version.txt)
모든 Optuna Trial 로그 및 모델은 'optuna_runs\v7' 아래에 저장됩니다.


In [3]:
def Transformer_Train(epoch, device, train_loader, val_loader, NN, loss_function, optimizer, scheduler_plateau,
                      warmup_epochs, log_dir, save_path, patience, trial=None):
    best_val_f1_weighted = 0.0
    no_improve_epochs = 0

    from torch.utils.tensorboard import SummaryWriter
    writer = SummaryWriter(log_dir)
    print(f"  [Trial {trial.number}] TensorBoard 로그 디렉토리: {log_dir}")

    if warmup_epochs > 0:
        initial_lr = optimizer.param_groups[0]['lr']
        warmup_lr_schedule = lambda e: (e + 1) / warmup_epochs if e < warmup_epochs else 1.0

    for e in range(1, epoch + 1):
        NN.train()
        total_loss = 0

        if warmup_epochs > 0 and e <= warmup_epochs:
            for param_group in optimizer.param_groups:
                param_group['lr'] = initial_lr * warmup_lr_schedule(e - 1)

        for batch_idx, (data, attention_mask, labels) in enumerate(train_loader):
            data, attention_mask, labels = data.to(device), attention_mask.to(device), labels.to(device)
            optimizer.zero_grad()
            predictions = NN(data, attention_mask)
            loss = loss_function(predictions, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_train_loss = total_loss / len(train_loader)

        NN.eval()
        val_preds = []
        val_true = []
        with torch.no_grad():
            for data, attention_mask, labels in val_loader:
                data, attention_mask, labels = data.to(device), attention_mask.to(device), labels.to(device)
                predictions = NN(data, attention_mask)
                val_preds.extend(torch.argmax(predictions, dim=1).cpu().numpy())
                val_true.extend(labels.cpu().numpy())

        val_acc = accuracy_score(val_true, val_preds)
        val_f1_weighted = f1_score(val_true, val_preds, average='weighted', zero_division=0)
        val_f1_macro = f1_score(val_true, val_preds, average='macro', zero_division=0)

        if e > warmup_epochs:
            scheduler_plateau.step(val_f1_weighted)

        if val_f1_weighted > best_val_f1_weighted:
            best_val_f1_weighted = val_f1_weighted
            no_improve_epochs = 0
        else:
            no_improve_epochs += 1

        print(f"  [Trial {trial.number}] Epoch {e}\tTrain Loss: {avg_train_loss:.5f}\tVal Acc: {val_acc:.4f}\tVal F1 (Weighted): {val_f1_weighted:.4f}\tVal F1 (Macro): {val_f1_macro:.4f}\tNo Improve Epochs: {no_improve_epochs}")

        writer.add_scalar('Loss/train', avg_train_loss, e)
        writer.add_scalar('Metrics/Val_Accuracy', val_acc, e)
        writer.add_scalar('Metrics/Val_F1_Weighted', val_f1_weighted, e)
        writer.add_scalar('Metrics/Val_F1_Macro', val_f1_macro, e)
        writer.add_scalar('Learning_Rate', optimizer.param_groups[0]['lr'], e)

        if trial:
            trial.report(val_f1_weighted, e)
            if trial.should_prune():
                print(f"Trial {trial.number} is pruned at epoch {e}.")
                raise optuna.exceptions.TrialPruned()

        if no_improve_epochs >= patience:
            print(f"조기 종료: {patience} 에포크 동안 성능 개선 없음. Trial {trial.number} 종료.")
            break
    
    writer.close()
    
    return best_val_f1_weighted

In [4]:
def objective(trial):
    # 1. 하이퍼파라미터 제안
    embedding_dim = trial.suggest_categorical('embedding_dim', [128, 256])
    hidden_dim = trial.suggest_categorical('hidden_dim', [128, 256, 512])
    n_layers = trial.suggest_int('n_layers', 2, 4)
    n_heads = trial.suggest_categorical('n_heads', [4, 8, 16])
    dropout_p = trial.suggest_float('dropout_p', 0.1, 0.5, step=0.1)
    learning_rate = trial.suggest_loguniform('learning_rate', 5e-6, 5e-5)
    warmup_epochs = trial.suggest_int('warmup_epochs', 5, 15)
    patience = trial.suggest_int('patience', 10, 20)
    use_class_weights = trial.suggest_categorical('use_class_weights', [True, False])

    if hidden_dim % n_heads != 0:
        raise optuna.exceptions.TrialPruned(f"hidden_dim ({hidden_dim}) is not divisible by n_heads ({n_heads})")

    # 2. 모델 인스턴스 생성 및 하이퍼파라미터 적용
    NN = Transformer(vocab_size=VOCAB_SIZE, embedding_dim=embedding_dim, hidden_dim=hidden_dim,
                     output_dim=OUTPUT_DIM, n_layers=n_layers, n_heads=n_heads, dropout_p=dropout_p,
                     max_len=MAX_LEN, pad_token_id=PAD_TOKEN_ID).to(device)

    class_weights = None
    if use_class_weights:
        all_train_labels_original = train_df['감정'].values.astype(int)
        num_classes = NN.output_dim
        label_counts_original = np.bincount(all_train_labels_original, minlength=num_classes)
        class_counts_tensor = torch.tensor(label_counts_original, dtype=torch.float)
        # 0으로 나누는 것을 방지 (매우 드물게 발생할 수 있는 0개 클래스 방지)
        class_counts_tensor = torch.where(class_counts_tensor == 0, torch.tensor(1.0), class_counts_tensor) 
        class_weights = (class_counts_tensor.sum() / class_counts_tensor).to(device)
        print(f"Trial {trial.number}: 계산된 클래스 가중치: {class_weights.tolist()}")
    else:
        print(f"Trial {trial.number}: 클래스 가중치 미사용.")


    # 4. 손실 함수 (클래스 가중치 적용 여부에 따라 초기화)
    if use_class_weights:
        loss_function = nn.CrossEntropyLoss(weight=class_weights)
    else:
        loss_function = nn.CrossEntropyLoss()

    # 5. 옵티마이저
    optimizer = optim.AdamW(NN.parameters(), lr=learning_rate)

    # 6. 스케줄러
    scheduler_plateau = ReduceLROnPlateau(optimizer, mode="max", factor=0.8, patience=5, min_lr=1e-7)

    # 7. 학습 실행 (Transformer_Train 함수 호출)
    log_dir = os.path.join(OPTUNA_VERSION_DIR, f"trial_{trial.number}_params_emb{embedding_dim}_heads{n_heads}_lr{learning_rate:.1e}_cw{use_class_weights}")
    save_path = os.path.join(OPTUNA_VERSION_DIR, f"model_trial_{trial.number}.pt")
    
    os.makedirs(log_dir, exist_ok=True)
    os.makedirs(os.path.dirname(save_path), exist_ok=True)

    val_f1_weighted = Transformer_Train(
        epoch=10000,
        device=device,
        train_loader=train_loader,
        val_loader=val_loader,
        NN=NN,
        loss_function=loss_function,
        optimizer=optimizer,
        scheduler_plateau=scheduler_plateau,
        warmup_epochs=warmup_epochs,
        log_dir=log_dir,
        save_path=save_path,
        patience=patience,
        trial=trial
    )

    return val_f1_weighted

In [7]:
# Optuna Study 저장 폴더 및 파일 경로 설정
OPTUNA_STORAGE_DIR = 'Optuna_Storage'
STUDY_SAVE_PATH = os.path.join(OPTUNA_STORAGE_DIR, 'optuna_study.pkl')

# Optuna 저장 폴더가 없다면 생성
os.makedirs(OPTUNA_STORAGE_DIR, exist_ok=True)

# Study 객체를 저장할 변수 미리 선언
study = None

# 이전에 저장된 Study가 있는지 확인하고 로드
if os.path.exists(STUDY_SAVE_PATH):
    print(f"이전 Study '{STUDY_SAVE_PATH}'를 로드합니다...")
    try:
        study = joblib.load(STUDY_SAVE_PATH)
        print(f"Study 로드 완료. 현재 {len(study.trials)}개의 트라이얼 기록이 있습니다.")
    except Exception as e:
        print(f"Study 로드 중 오류 발생: {e}. 새로운 Study를 생성합니다.")
        # 로드 실패 시 새로운 Study 생성
        study = optuna.create_study(
            direction="maximize",
            sampler=optuna.samplers.TPESampler(),
            pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=30, interval_steps=10)
        )
else:
    print(f"저장된 Study 파일이 없습니다. 새로운 Study를 생성합니다.")
    # 저장된 파일이 없다면 새로운 Study 생성
    study = optuna.create_study(
        direction="maximize",
        sampler=optuna.samplers.TPESampler(),
        pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=30, interval_steps=10)
    )

print("Optuna 최적화 시작...")
# 최적화 실행
# n_trials는 추가로 실행할 횟수를 의미합니다.
# 만약 이미 5개의 트라이얼을 실행했고, 총 10개를 하고 싶다면 n_trials=5로 다시 호출합니다.
study.optimize(objective, n_trials=5)

이전 Study 'Optuna_Storage\optuna_study.pkl'를 로드합니다...
Study 로드 완료. 현재 20개의 트라이얼 기록이 있습니다.
Optuna 최적화 시작...
Trial 20: 계산된 클래스 가중치: [15.832883834838867, 18.12030792236328, 9.509984016418457, 12.201488494873047, 1.8166542053222656, 12.491228103637695, 15.618878364562988]
  [Trial 20] TensorBoard 로그 디렉토리: optuna_runs\v6\trial_20_params_emb256_heads16_lr4.9e-05_cwTrue


C:\Users\Administrator\AppData\Local\Temp\ipykernel_16144\2319572601.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 5e-6, 5e-5)


  [Trial 20] Epoch 1	Train Loss: 1.82967	Val Acc: 0.2609	Val F1 (Weighted): 0.2910	Val F1 (Macro): 0.2197	No Improve Epochs: 0
  [Trial 20] Epoch 2	Train Loss: 1.73152	Val Acc: 0.2925	Val F1 (Weighted): 0.3144	Val F1 (Macro): 0.2487	No Improve Epochs: 0
  [Trial 20] Epoch 3	Train Loss: 1.62947	Val Acc: 0.3931	Val F1 (Weighted): 0.4203	Val F1 (Macro): 0.3187	No Improve Epochs: 0
  [Trial 20] Epoch 4	Train Loss: 1.54278	Val Acc: 0.3859	Val F1 (Weighted): 0.4118	Val F1 (Macro): 0.3357	No Improve Epochs: 1
  [Trial 20] Epoch 5	Train Loss: 1.46882	Val Acc: 0.3515	Val F1 (Weighted): 0.3751	Val F1 (Macro): 0.3273	No Improve Epochs: 2
  [Trial 20] Epoch 6	Train Loss: 1.40667	Val Acc: 0.3855	Val F1 (Weighted): 0.4078	Val F1 (Macro): 0.3463	No Improve Epochs: 3
  [Trial 20] Epoch 7	Train Loss: 1.34306	Val Acc: 0.4343	Val F1 (Weighted): 0.4572	Val F1 (Macro): 0.3693	No Improve Epochs: 0
  [Trial 20] Epoch 8	Train Loss: 1.29837	Val Acc: 0.4411	Val F1 (Weighted): 0.4651	Val F1 (Macro): 0.3868	No Im

[I 2025-07-11 16:09:58,479] Trial 20 pruned. 


  [Trial 20] Epoch 30	Train Loss: 0.69455	Val Acc: 0.4688	Val F1 (Weighted): 0.4958	Val F1 (Macro): 0.4123	No Improve Epochs: 19
Trial 20 is pruned at epoch 30.
Trial 21: 클래스 가중치 미사용.
  [Trial 21] TensorBoard 로그 디렉토리: optuna_runs\v6\trial_21_params_emb256_heads4_lr6.2e-06_cwFalse


...

  [Trial 21] Epoch 30	Train Loss: 1.23955	Val Acc: 0.5951	Val F1 (Weighted): 0.5154	Val F1 (Macro): 0.2863	No Improve Epochs: 0
Trial 21 is pruned at epoch 30.
Trial 22: 클래스 가중치 미사용.
  [Trial 22] TensorBoard 로그 디렉토리: optuna_runs\v6\trial_22_params_emb256_heads16_lr8.1e-06_cwFalse


C:\Users\Administrator\AppData\Local\Temp\ipykernel_16144\2319572601.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 5e-6, 5e-5)


  [Trial 22] Epoch 1	Train Loss: 1.77414	Val Acc: 0.5504	Val F1 (Weighted): 0.3908	Val F1 (Macro): 0.1014	No Improve Epochs: 0
  [Trial 22] Epoch 2	Train Loss: 1.60137	Val Acc: 0.5504	Val F1 (Weighted): 0.3908	Val F1 (Macro): 0.1014	No Improve Epochs: 1
  [Trial 22] Epoch 3	Train Loss: 1.57241	Val Acc: 0.5504	Val F1 (Weighted): 0.3908	Val F1 (Macro): 0.1014	No Improve Epochs: 2
  [Trial 22] Epoch 4	Train Loss: 1.53597	Val Acc: 0.5504	Val F1 (Weighted): 0.3908	Val F1 (Macro): 0.1014	No Improve Epochs: 3
  [Trial 22] Epoch 5	Train Loss: 1.49092	Val Acc: 0.5504	Val F1 (Weighted): 0.3948	Val F1 (Macro): 0.1068	No Improve Epochs: 0
  [Trial 22] Epoch 6	Train Loss: 1.46383	Val Acc: 0.5522	Val F1 (Weighted): 0.4197	Val F1 (Macro): 0.1406	No Improve Epochs: 0
  [Trial 22] Epoch 7	Train Loss: 1.44205	Val Acc: 0.5520	Val F1 (Weighted): 0.4183	Val F1 (Macro): 0.1420	No Improve Epochs: 1
  [Trial 22] Epoch 8	Train Loss: 1.42671	Val Acc: 0.5526	Val F1 (Weighted): 0.4192	Val F1 (Macro): 0.1436	No Im

[W 2025-07-11 16:38:45,181] Trial 22 failed with parameters: {'embedding_dim': 256, 'hidden_dim': 128, 'n_layers': 3, 'n_heads': 16, 'dropout_p': 0.4, 'learning_rate': 8.088945618561149e-06, 'warmup_epochs': 11, 'patience': 13, 'use_class_weights': False} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "C:\Users\Administrator\anaconda3\envs\tensor\Lib\site-packages\optuna\study\_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\Administrator\AppData\Local\Temp\ipykernel_16144\2319572601.py", line 54, in objective
    val_f1_weighted = Transformer_Train(
                      ^^^^^^^^^^^^^^^^^^
  File "C:\Users\Administrator\AppData\Local\Temp\ipykernel_16144\2591231837.py", line 28, in Transformer_Train
    optimizer.step()
  File "C:\Users\Administrator\anaconda3\envs\tensor\Lib\site-packages\torch\optim\optimizer.py", line 485, in wrapper
    out = func(*args, **kwa

KeyboardInterrupt: 

> tensorboard --logdir optuna_runs

In [5]:
# 최적화가 끝난 후 Study 객체 저장
print(f"최적화 결과를 '{STUDY_SAVE_PATH}'에 저장합니다...")
joblib.dump(study, STUDY_SAVE_PATH)
print("Study 저장 완료.")


print("\n--- Optuna 최적화 결과 ---")
print(f"최적의 하이퍼파라미터 조합 (Best Trial): {study.best_trial.params}")
print(f"최적의 검증 Weighted F1 (Best Value): {study.best_trial.value:.4f}")

print("\n--- 모든 시도 결과 ---")
for i, trial in enumerate(study.trials):
    print(f"Trial {i}: Value={trial.value:.4f}, Params={trial.params}, State={trial.state}")

NameError: name 'STUDY_SAVE_PATH' is not defined

In [7]:
OPTUNA_STORAGE_DIR = 'Optuna_Storage'
STUDY_SAVE_PATH = os.path.join(OPTUNA_STORAGE_DIR, 'optuna_study.pkl')

# Optuna 저장 폴더가 없다면 생성
os.makedirs(OPTUNA_STORAGE_DIR, exist_ok=True)

# Study 객체를 저장할 변수 미리 선언
study = None
if os.path.exists(STUDY_SAVE_PATH):
    print(f"이전 Study '{STUDY_SAVE_PATH}'를 로드합니다...")
    try:
        study = joblib.load(STUDY_SAVE_PATH)
        print(f"Study 로드 완료. 현재 {len(study.trials)}개의 트라이얼 기록이 있습니다.")
    except Exception as e:
        print(f"Study 로드 중 오류 발생: {e}. 새로운 Study를 생성합니다.")
        # 로드 실패 시 새로운 Study 생성
        study = optuna.create_study(
            direction="maximize",
            sampler=optuna.samplers.TPESampler(),
            pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=30, interval_steps=10)
        )
print("\n--- Optuna 최적화 결과 ---")
print(f"최적의 하이퍼파라미터 조합 (Best Trial): {study.best_trial.params}")
print(f"최적의 검증 Weighted F1 (Best Value): {study.best_trial.value:.4f}")

print("\n--- 모든 시도 결과 ---")
for i, trial in enumerate(study.trials):
    print(f"Trial {i}: Value={trial.value:.4f}, Params={trial.params}, State={trial.state}")

이전 Study 'Optuna_Storage\optuna_study.pkl'를 로드합니다...
Study 로드 완료. 현재 20개의 트라이얼 기록이 있습니다.

--- Optuna 최적화 결과 ---
최적의 하이퍼파라미터 조합 (Best Trial): {'embedding_dim': 256, 'hidden_dim': 128, 'n_layers': 4, 'n_heads': 8, 'dropout_p': 0.4, 'learning_rate': 4.565359362828951e-05, 'warmup_epochs': 5, 'patience': 20, 'use_class_weights': False}
최적의 검증 Weighted F1 (Best Value): 0.6540

--- 모든 시도 결과 ---
Trial 0: Value=0.6028, Params={'embedding_dim': 128, 'hidden_dim': 128, 'n_layers': 2, 'n_heads': 4, 'dropout_p': 0.5, 'learning_rate': 1.1933217143832069e-05, 'warmup_epochs': 11, 'patience': 11, 'use_class_weights': False}, State=1
Trial 1: Value=0.6395, Params={'embedding_dim': 128, 'hidden_dim': 256, 'n_layers': 2, 'n_heads': 16, 'dropout_p': 0.1, 'learning_rate': 2.9579485662059165e-05, 'warmup_epochs': 10, 'patience': 20, 'use_class_weights': False}, State=1
Trial 2: Value=0.6394, Params={'embedding_dim': 128, 'hidden_dim': 128, 'n_layers': 2, 'n_heads': 16, 'dropout_p': 0.30000000000000004, 'le

TypeError: unsupported format string passed to NoneType.__format__